## 06. PyTorch Transfer Learning

#### **Transfer learning**

=> involves taking the parameters of what one model has learned on another dataset and applying to our own problem.

- Pretrained model = foundation model

=> e.g. take the patterns a computer vision model has learned from datasets such as `ImageNet`, and use them to power our FoofVision Mini model.

#### Why **Transfer learning**?

- (1) can leverage an existing model (usually a neurl network architecture) proven to work on problems similar to our own
- (2) can leverage a working model which has already learned patterns on similar data to our own. This often results in achieving great results with less custom data.

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.1.0.dev20230709
0.16.0.dev20230709


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
from go_modular import data_setup, engine

device = "gpu" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_built() else "cpu"
device

'mps'

### 1. Get Data

In [3]:
import os
import zipfile
from pathlib import Path
import requests

# Setup data path
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi" # images from a subset of classes from the Food101 dataset

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists, skipping re-download.")
else:
    print(f"Did not find {image_path}, downloading it...")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

    # Remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi directory exists, skipping re-download.


In [4]:
# Setup train and test directory path
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

### 2. Create datasets and dataloaders using script

=> use `data_setup.py` and `create_dataloaders()` function made in notebook05.

=> How to `transform` it?
- (1) manually create transforms -> define what transforms you want you dara to go through
- (2) automatically created transforms -> defined by the model you'd like to use

**Note:** the data that pass through the pre-trained model is `transformed` in the same way that the model was trined on

#### 2.1 **Manually** create a transform for `torchvision.models`

In [5]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize   # make sure images have the same distribution as ImageNet
                # (where our pretrained models have been trained)
])

In [6]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transform,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x104f6be80>,
 ['pizza', 'steak', 'sushi'])

#### 2.2 **Auto** create transform for `torchvision.models`

=> As of `torchvision v0.13+` there is now support for automatic data transform creation based on the pretrained model weights you're using.

In [7]:
# Get a set of pretrained model weights
# "DEFAULT" = best available weights
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [8]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [9]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transforms,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x104f69810>,
 ['pizza', 'steak', 'sushi'])